# EDA on Data for Academic Interns in UC Berkeley's CS61A Course

## Imports

In [ ]:
# necessary imports
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import re

## EDA and Preprocessing

I start by reading in the scraped data into a pandas DataFrame.

In [ ]:
ai_61a_df = pd.read_csv("ai_61a_data.csv")
ai_61a_df.head()

Next, I want a quick overview of the DataFrame.

In [ ]:
ai_61a_df.describe()

Confirming the $6$ unique semesters and the number of AIs hired:

In [ ]:
ai_61a_df['semester'].value_counts()

It looks good, it matches the data that I have in `scraper.ipynb`. Something interesting is that for the most part, the number of AIs recruited each semester seems to be increasing, except for the slight dip from `fa22`'s $152$ AIs to `sp23`'s $130$. But thinking more about this, I'm not sure why there would be a decrease. I graduated in Fall of 2022, and I know during that semester a strike occurred that was largely organized by CS students to demand for more funding so that more people could be hired. From the plethora of strike status update emails I received, I recall that the strike was *somewhat* successful and the graduate and undergraduate students received slightly better pay and more benefits, so how did AI hiring numbers go down?

My assumption is maybe that since AIs are *not paid*, it doesn't matter if funding increased or not, the number of opportunities should stay roughly the same. Maybe, since there was a pay increase, the department was able to hire more *tutors* and *TAs* (the roles that are higher in the pyramid than AIs), and so the course didn't have to hire as many AIs. But at the same time, I don't see a reason not to hire *more* AIs. Office Hour queues are an absolute nightmare, especially during project weeks, so more AIs would only help... right?

On a second thought, I'm not entirely sure if 61A AIs help with projects; I think the policy changes each semester depending on the professor and the AI Coordinator. I know 61B AIs definitely help with projects (Gitlet and Enigma... never again, thank god Hilfinger retired). Regardless, it seems like my initial theories here are inconclusive and I'll need to physically go back and see to get an answer.


Going back to the overiew, assuming everyone has a name, there were a total of $643$ AIs scraped from the $6$ semesters of AI pages for CS61A. $637$ of them are unique, meaning some were rehired. This is very interesting, since I had a friend tell me that 61A AIs are *very rarely rehired*. Looking at the `freq` row from the results of `describe()`, it seems like no one was rehired for more than $2$ semesters, once again reinforcing my friend's claims. Now I'm curious to see how who rehired:

In [ ]:
# Checking who got rehired for more than one semester
ai_61a_df['name'].value_counts().head(10)

Ok Justin Zhang, Matthew Ybarra, Andrew Wang, Bryan Li, Andy Chen, and Wonjae Lee. The big brain ones I see...

Wait, Wonjae Lee! I remember imputing his pronouns during data scraping, and I had to assume his gender. Let's check if I got his pronouns correct:

In [ ]:
# Filter for Wonjae Lee
ai_61a_df[ai_61a_df['name']=='Wonjae Lee']

I imputed his `fa21` pronouns, and it seems I did assume correctly since his `sp21` are the same as my imputed values. It also seems like the rehire occured in consecutive semesters, let's see if that is also the case for the other $5$ rehired AIs.

In [ ]:
# Filter is going to look really ugly, but it is what it is
ai_61a_df[(ai_61a_df['name']=='Justin Zhang') | (ai_61a_df['name']=='Matthew Ybarra') | (ai_61a_df['name']=='Andrew Wang') |
          (ai_61a_df['name']=='Bryan Li') | (ai_61a_df['name']=='Andy Chen')].sort_values('name')

Very interesting. Like Wonjae, everyone else seems to be a rehire *except* for Bryan Li. From reading their bios, the numbers don't add up, as one was a freshman in `sp23` and the other was a sophomore in `fa21`, not to mention their majors are completely different. Unfortunately there's no unique identifier that I could scrape from the AI pages, so issues like these will occur regardless. Nevertheless, this shouldn't negatively affect the question I'm trying to answer.

Admittedly, looking at this makes me a bit salty. I was a 61A AI in Fall of 2021, but was not rehired for Spring of 2022.

Proof:

In [ ]:
ai_61a_df[ai_61a_df['name']== 'Daniel Huang']

Whatever, I'm on to greener pastures (sorta).


Returning to the overview, I see that there are $626$ listed pronouns, meaning the DataFrame missing $643-626=17$ pronoun entries. Let's confirm that:

In [ ]:
print("Num Empty Pronouns: {}".format(sum(ai_61a_df['pronoun'].isna())))

Pronouns are ultimately what I will be using to determine gender, which is the crux of this project's question. Taking a closer look at these $17$ entries:

In [ ]:
ai_61a_df[ai_61a_df['pronoun'].isna()]

I could try to manually input pronouns for these entries by looking at their name, reading their bio, and looking up their photo in their corresponding semester's AI page. $17$ entries however is roughly only $3\%$ of my total data, I think I'm fine with losing $3\%$ of my data and not having to assume someone's gender. I perform the null removal in the following cell:

In [ ]:
# remove rows with null pronouns
ai_61a_df = ai_61a_df[~ai_61a_df['pronoun'].isna()]

# check removal
ai_61a_df.describe()

Removal looks successful. I want to conduct some more exploration, like determining the proportion of male to female AIs and seeing how the proportion changes when grouped by semester. But to do all this I'm going to have to move on to feature engineering.

## Feature Engineering and Analysis

I want to start creating a new feature `gender` with my `pronoun` column, more specifically, I'm going to create a mapping:
* 'he/him/his' -> 'male'
* 'she/her/hers' -> 'female'
* everything else -> 'other'

But to perform the third mapping, I'm going to have to see what "everything else" consists of:

In [ ]:
# Filter for pronouns falling under "everything else"
ai_61a_df[~((ai_61a_df['pronoun']=='he/him/his') | (ai_61a_df['pronoun']=='she/her/hers'))]

I see comrade makes a return...

On second thought, there are only $7$ entries here (I might've unintentionally created a diversity indicator for AIs in Berkeley's CS department...); there shouldn't be a harm in removing them. At the end of the day, my question doesn't really concern this subset of people, so their data wouldn't have been used either way. I proceed to remove them:

In [ ]:
# Remove rows with non-binary pronouns
ai_61a_df = ai_61a_df[(ai_61a_df['pronoun']=='he/him/his') | (ai_61a_df['pronoun']=='she/her/hers')]

Removal looks successful, $626-7=619$. Now, time to proceed to the mapping. To do so, I use a dictionary with my desired mapping and the pandas function [`pandas.Series.map`](https://pandas.pydata.org/docs/reference/api/pandas.Series.map.html).

At the end of the day, this change is just changing one set of strings to another set of strings, the data type doesn't even change. I'm still going to do it though. Personally, it's just easier for me to work with my data this way.

In [ ]:
# Establish mapping
pronoun_to_gender_dict = {"he/him/his":"male", "she/her/hers": "female"}

# perform replacement
ai_61a_df['gender'] = ai_61a_df['pronoun'].map(pronoun_to_gender_dict)
ai_61a_df

With this new feature added, I can take a look at the proportion of male to female students and compute the percentage difference using a formula from this [site](https://www.omnicalculator.com/math/percentage-difference). Additionally, I decided to use [`seaborn.barplot`](https://seaborn.pydata.org/generated/seaborn.barplot.html) to help with visualization.

In [ ]:
# Python function that takes in a DataFrame and a binary group label
# Will print the count proportion of each group, compute and print the
# percentage difference between the two values, and display a barplot visualization
# Returns a tuple containing the group proportions in alphabetical order and the percent diff
# Example: if `binary_group=gender`, then the 0th element is `female` proportion
# and the 1st element would be `male` proportion, and third element is percent diff
def compute_proportion_and_visualize(df, binary_group):

    # Helper function to compute percentage difference with formula from:
    # https://www.omnicalculator.com/math/percentage-difference
    # Takes in a x and y and calculates the percentage difference of those two values
    def percent_difference(x, y):
        percent_diff = abs(x - y) / ((x + y)/2)*100
        return percent_diff

    
    # compute proportions and display thereof
    binary_group_series = df.groupby(binary_group).size()
    group_0_proportion = binary_group_series.values[0]/sum(binary_group_series.values)
    group_1_proportion = binary_group_series.values[1]/sum(binary_group_series.values)
    print("{} proportion: {:.2f}".format(binary_group_series.index[0], group_0_proportion))
    print("{} proportion: {:.2f}".format(binary_group_series.index[1], group_1_proportion))

    # call helper percent_difference to display percentage difference
    percent_diff = percent_difference(group_0_proportion, group_1_proportion)
    print("percentage difference: {:.0f}%".format(percent_diff))

    # visualize with seaborn.barplot
    plt.title('Barplot of {} Counts'.format(binary_group_series.index.name))
    sns.barplot(x=binary_group_series.index, y=binary_group_series.values)
    plt.ylabel('Count')
    plt.show()

    # return the tuple of group proportions
    return (group_0_proportion, group_1_proportion, percent_diff)

In [ ]:
# Call compute_proportion_and_visualize
overall_female_proportion, overall_male_proportion, overall_percent_diff = compute_proportion_and_visualize(ai_61a_df, 'gender')

Well then, if this isn't already pretty clear. There is clearly a higher proportion of male to female AIs in CS61A. A proportion of $0.37$ females to $0.63$, resulting in a percentage difference of $51\%$.

I want to see how this data will change when grouped by semester, `sem`. Has the percetage difference changed over the semesters, or has it largely stayed the same? To answer the chronological aspect of this question, I will use a manually ordered list of available semesters:

`sem_list = ['sp21', 'fa21', 'sp22', 'su22', 'fa22', 'sp23']`

In [ ]:
# instantiate chronologically correct semester list
sem_list = ['sp21', 'fa21', 'sp22', 'su22', 'fa22', 'sp23']

# Python function that takes in a pandas DataFrame `df`, the name of the filter `filter_feature`
# a list of all the possible filters `filter_list`, and lastly a binary_group to past into 
# `compute_proportion_and_visualize`. This previously implemented function is called on each
# filter group with the passed in `binary_group`
# Returns a list containing the tuples of each proportion pair. Number of pairs is dependent on
# the number of filter_groups in filter_list.
def bar_analysis_summary(df, filter_feature, filter_list, binary_group):

    # instantiate list to append to
    proportion_pair_list = []
    
    # loop through each filter group
    for filter_group in filter_list:
        curr_df = df[df[filter_feature]==filter_group] # filter for the current filter group in the loop
        
        # print time of current filter group
        print("time: {}".format(filter_group))
        
        # call `compute_proportion_and_visualize` on curr_df and save the tuple
        curr_tuple = compute_proportion_and_visualize(curr_df, binary_group)

        # append curr_tuple to list
        proportion_pair_list.append(curr_tuple)
        print('-----------------------\n') # formatting

    # return final list
    return proportion_pair_list

In [ ]:
# Call bar_analysis
sem_gender_proportion_tuple_list = bar_analysis_summary(ai_61a_df, 'semester', sem_list, 'gender')

Interesting. It is clearly the case the proportion of male AIs have always been greater than that of female AIs. Since I'm dealing with time, I want to visualize with a [lineplot](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.plot.html). I want to see how each proportion and the percentage difference has changed over time.

To do this though I need to somehow convert my semester values in `sem_list` into a scalar, since lineplots cannot take in non-scalar values. To do so, I propose the following mapping:

`['sp21', 'fa21', 'sp22', 'su22', 'fa22', 'sp23']` -> `[0, 1, 2, 3, 4, 5]`

Simple, but it should work.

In [ ]:
scalar_sem_list = [0, 1, 2, 3, 4, 5]

# Function that displays a lineplot of change in proportion and percentage diff
# of a binary_group in df. df and binary_group are passed in for display header purposes
# it's needed to properly format title and labels
def chrono_analysis(df, binary_group, proportion_tuple_list, time):
    # get proper labels
    binary_group_series = df.groupby(binary_group).size()
    group_0_label = binary_group_series.index[0]
    group_1_label = binary_group_series.index[1]

    # instantiate empty lists to store data for plotting
    chrono_group_0 = []
    chrono_group_1 = []
    chrono_proportion_diff = []

    for tuple in proportion_tuple_list:
        ### tuple structure: ###
        # 0: group 0 proportion
        # 1: group 1 proportion
        # 2: percentage difference

        # append the data as a list pair to the previously instantiated lists
        chrono_group_0.append(tuple[0]) # group 0 proportions
        chrono_group_1.append(tuple[1]) # group 1 proportions
        chrono_proportion_diff.append(tuple[2]/100) # percentages, divide by 100 to get proportion

    # # all data properly created, plot
    plt.title("Proportion and Proportion Difference in {} Over Time".format(binary_group_series.index.name))
    plt.plot(time, chrono_group_0, color='red', label=group_0_label)
    plt.plot(chrono_group_1, color='blue', label=group_1_label)
    plt.plot(chrono_proportion_diff, color='green', label='difference')
    #plt.grid()
    plt.legend()
    plt.xlabel('Time')
    plt.ylabel('Proportion')
    plt.ylim(bottom=0) # want to show origin point
    plt.show()    

In [ ]:
# call chrono_analysis
chrono_analysis(ai_61a_df, 'gender', sem_gender_proportion_tuple_list, scalar_sem_list)

From the plot, it appears that gender diversity has largely stayed stagnant, if not gotten worse, in recent semesters. There was a slight dip in difference proportion at time $4$, Fall 2022 semester, but the male AI proportion quickly bounced back in the most recent semester, Spring 2023.

How do you increase women representation in CS? I'm sure this is a complex problem, and I don't think I know nearly enough about it to warrant an educated opinion. I guess there's a reason why there are programs and scholarships more widely available for women stem.

Regardless, finally moving on to my main question.

## Feature Engineering and Analysis (Main Question)

### Project Motivations and Hypothesis

Over the years, with my various interactions with my fellow peers and students, I've come to notice that women are more likely to send messages in *all lowercase*. Why does this happen? My personal theory is as follows:

Women write in all lowercase because:

1. Gives off cute e-girl doki-doki uwu vibe
2. Relays a feeling of gentleness, giving them a *softer* tone, ultimately making them seem more approachable and nice

I've share this observation of mine with some friends, some have agreed but some also claim to not have noticed or don't think that's the case; hence born the motivation for this project. I want to show that from just the CS61A AI page alone, the girl AIs are more likely to use lowercase to communicate than the guys.

But the page is in an academic setting, and writing messages in an **all** lowercase form is very rare. No, this is not an excuse for me to fall back on in case my theory fails. The reason why it's rare is because classes are often capitalized, like CS61A. Immediately this would destroy the *all* quantifier of my statement. In addition, majors names are often capitalized or abbreviated with their initials, like CS and EECS. Thus, to actually make this project worth the effort I've put in so far, I want to propose a modified version of the previous statement, presented in the form of a hypothesis:

**Hypothesis:** Women are more likely to *not use* proper case in messages than their male counterparts.

There's a change in diction here, *lowercase* vs *proper case*. In a way this kind of changes the argument, but I would argue they manifest in the same way. Outside of proper nouns and abbreviations, capital letters are only ever used at the beginning of a sentence, so commonly after a period. Thus, my hypothesis is saying that since women are more likely to use lowercase, they are also less liekly to be using proper case.

So how would I test for this? I do not know of any traditional method that I have learned in school that would do this, maybe because a question such as this probably really isn't even worth answering since it provides no benefit to society. A/B testing? Modeling? None of these apply. Machine learning to determine message patterns in women vs men, yeah definitely possible and probably the better way to approach this project, but that's beyond my knowledge and probably not worth the effort (like I said, zero benefit to society).

So I'm just going to approach tackle this question my own way. By no means is this going to be a definitive, undisputable answer, it's just going to be evidence that may or may not lean towards my hypothesis. With that said, I am going to extrapolate, or technically in this case feature engineer, the *letter case percentage composition* of all the AIs I have scraped from CS61A.

What is *letter case percentage composition*? I made it up. Previously, I was thinking of just calling the percentage of lowercase letters in women's messages, but that is just way to verbose and it doesn't sound smart. So I racked what little passion I have for chemistry left (passionate in highschool, now the subject is dead to me) and looked to [wikipedia for inspiration](https://en.wikipedia.org/wiki/Letter_case) and arrived at this term. Thus, its formal definition:

**letter case percentage composition**

*noun*

1. in the context of a message, the manner of being composed; makeup; constitution; quantified in percentage of upper and lowercase letters.

In layman's terms, given a message, what is the percentage of upper and lowercase letters.

With all that said, let me get started on just that.

### Feature Engineering - Letter Case Percentage Composition

To compute percentage composition, I need to get a character count for each AI's bio. But before I do that, I need to clean their bio up a little bit more, since it is being inflated by whitespace characters and punctuation. I can do that quite simply with regex, specifically [`pandas.Series.str.replace`](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.replace.html):

In [ ]:
# pattern of characters to remove in bio
remove_pattern = "[\s.,'!?;:()-]"

# perform removal
ai_61a_df['clean_bio'] = ai_61a_df['bio'].str.replace(remove_pattern, '', regex=True)
ai_61a_df.head(5)

Unfortunately the removal is not complete. AIs are young people, and many of them use emojis and innovative other what-nots that I'm too boomer to keep up with. Thus, it is inevitable that there will be a little bit of character bloat. Nonetheless, I do believe that my removal pattern is comprehensive enough that the bloat retained is minimal, so I will continue to proceed.

Finally, the most important part of the project; determine the percentage of uppercase letters; lowercase can found in a similar way, and since I'm dealing with percentage, might as well get the total character count as well.

To start, I create the following three features that will be used to help me determine percentage composition

1. `letter_count`: num letters in the clean bio
2. `uppercase_count`: num uppercase letters in the clean bio
3. `lowercase_count`: num lowercase letters in the clean bio

In [81]:
# append feature #1
# this is essentially treating `clean_bio` as a String and calling `str.len` to get the length
ai_61a_df['letter_count'] = ai_61a_df['clean_bio'].apply(lambda clean_bio: len(str(clean_bio)))

# append feature #2
# this is a pretty complicated function, but I use the pandas.Series.apply function
# and then create a lambda function that takes in the column `clean_bio`
# which gets converted into a String, and as a String, I loop through it in a list comprehension, creating  
# a list that contains the value 1 if a letter is upper and 0 othwerise; the list gets summed up at the end
# essentially becoming a count for uppercase letters.
ai_61a_df['uppercase_count'] = ai_61a_df['clean_bio'].apply(lambda clean_bio: sum([1 if letter.isupper() else 0 for letter in str(clean_bio)]))

# append feature #3
# essentially the same as above but uses `str.islower` instead of `str.isupper``
ai_61a_df['lowercase_count'] = ai_61a_df['clean_bio'].apply(lambda clean_bio: sum([1 if letter.islower() else 0 for letter in str(clean_bio)]))

# project `clean_bio` and new features as a check
ai_61a_df[['clean_bio', 'letter_count', 'uppercase_count', 'lowercase_count']].head()


,clean_bio,letter_count,uppercase_count,lowercase_count
0,HeyImAakarshandImfromSanDiegoFreshmanstudyingE...,222,14,205
1,YowhatsupImAarnavafirstyearEECSstudentfromtheB...,174,12,160
2,HiImAaronanintendedCSmajorfromSouthKoreaItsmyf...,134,15,117
3,HiImAaryanafreshmanfromtheDetroitAreastudyingE...,117,11,106
4,HelloMynameisAbtinandIamafreshmanfromSanJosest...,143,11,132


Notice that it is not necessarily the case that `letter_count` $=$ `uppercase_count` $+$ `lowercase_count`. This is unsurprising; I mentioned the removal pattern was not comprehensive, and this is fine.

With these three features, I can easily compute the *letter case percentage composition:

* `lowercase_percentage` $=$ `lowercase_count` $/$ `letter_count`
* `uppercase_percentage` $=$ `uppercase_count` $/$ `letter_count`

In [83]:
# append lowercase_percentage
ai_61a_df['lowercase_percentage'] = ai_61a_df['lowercase_count']/ai_61a_df['letter_count']

# append uppercase_percentage
ai_61a_df['uppercase_percentage'] = ai_61a_df['uppercase_count']/ai_61a_df['letter_count']

# project `clean_bio`, `gender`, and new features as a check
ai_61a_df[['gender', 'clean_bio', 'lowercase_percentage', 'uppercase_percentage']].head()

,gender,clean_bio,lowercase_percentage,uppercase_percentage
0,male,HeyImAakarshandImfromSanDiegoFreshmanstudyingE...,0.923423,0.063063
1,male,YowhatsupImAarnavafirstyearEECSstudentfromtheB...,0.919540,0.068966
2,male,HiImAaronanintendedCSmajorfromSouthKoreaItsmyf...,0.873134,0.111940
3,male,HiImAaryanafreshmanfromtheDetroitAreastudyingE...,0.905983,0.094017
4,male,HelloMynameisAbtinandIamafreshmanfromSanJosest...,0.923077,0.076923


Alright. The moment of truth. Time to see the distribution of `uppercase_percentage` grouped by `gender`: